# Setup

In [ ]:
import numpy as np
import pandas as pd

# Data Preparation

## Data Loading

Grab the participants data; uncomment to run, it can be quite time consuming

In [ ]:
# %%bash

# cp "/work/cadotte_lab/bids_cmri/participants.tsv" "./participants.tsv"

Load the data with pandas, setting patient GRP as the index

In [ ]:
participants_df = pd.read_csv('participants.tsv', sep='\t')
participants_df = participants_df.set_index('GRP')
participants_df.shape

## mJOA Cleanup

Isolate the initial and 1-year mJOA metrics so they are not deleted by mistake 

In [ ]:
mjoa_cols = [
    "('mJOA', 'initial')",
    "('mJOA', '12 months')",
    "('mJOA; Total [CSA]', 'initial')",
    "('mJOA; Total [CSA]', '12 months')"
]
mjoa_df = participants_df.loc[:, mjoa_cols]

Transfer the CSA dataset's values into the original's when there is not an original value to go off of

In [ ]:
missing_idx = mjoa_df.loc[:, "('mJOA', 'initial')"].isna()
print(f"Before: {np.sum(missing_idx)}")
mjoa_df.loc[missing_idx, "('mJOA', 'initial')"] = mjoa_df.loc[missing_idx, "('mJOA; Total [CSA]', 'initial')"]
missing_idx = mjoa_df.loc[:, "('mJOA', 'initial')"].isna()
print(f"After: {np.sum(missing_idx)}")

In [ ]:
missing_idx = mjoa_df.loc[:, "('mJOA', '12 months')"].isna()
print(f"Before: {np.sum(missing_idx)}")
mjoa_df.loc[missing_idx, "('mJOA', '12 months')"] = mjoa_df.loc[missing_idx, "('mJOA; Total [CSA]', '12 months')"]
missing_idx = mjoa_df.loc[:, "('mJOA', '12 months')"].isna()
print(f"After: {np.sum(missing_idx)}")

In [ ]:
mjoa_df.shape

Drop the (now redundant) columns in both datasets

In [ ]:
mjoa_df = mjoa_df.drop(["('mJOA; Total [CSA]', 'initial')", "('mJOA; Total [CSA]', '12 months')"], axis=1)
participants_df = participants_df.drop(mjoa_cols, axis=1)

## Timepoint isolation

Isolate data w/o a timepoint before proceeding

In [ ]:
non_timed = participants_df.iloc[:, -20:]
timed = participants_df.drop(non_timed.columns, axis=1)

In [ ]:
non_timed.shape

In [ ]:
timed.shape

Only keep values w/ an initial time point; only mJOA is needed after 1 year, as it's the only value important to calculating the target, and it was handled during mJOA parsing prior.

In [ ]:
keep_cols = []
for c in timed.columns:
    if c.split(',')[1] == " 'initial')":
        keep_cols.append(c)

cleaned_participants_df = participants_df.loc[:, keep_cols]
cleaned_participants_df.loc[:, non_timed.columns] = non_timed
cleaned_participants_df.shape

Drop redundant columns in the dataset

In [ ]:
cleaned_participants_df = cleaned_participants_df.drop(columns=["('Surgical', 'initial')", "('BMI', 'initial')"])
cleaned_participants_df.shape

Reformat column headers to be cleaner, namely by removing the (now redundant) time point

In [ ]:
cols = [c.replace("'initial'", "") for c in cleaned_participants_df.columns]
cleaned_participants_df.columns = cols
cleaned_participants_df.shape

## EQ5D Unusual Null Value Correction

EQ5D occasionally uses the value of `4` to indicate a null value for some reason

In [ ]:
for c in cleaned_participants_df.columns:
    if 'EQ5D' in c:
        cleaned_participants_df.loc[cleaned_participants_df[c] == 4, c] = np.nan

## Erroneous BMI Correction

In [ ]:
cleaned_participants_df.loc[cleaned_participants_df['BMI'] == 0, 'BMI'] = np.nan

## Consolidation and Clean-Up

Add back in the mJOA metrics

In [ ]:
final_participants_df = cleaned_participants_df.copy()
final_participants_df.loc[:, mjoa_df.columns] = mjoa_df
final_participants_df.shape

Format the column headers to be cleaner by removing characters which could cause issues with common storage methodologies (namely CSV, TSV, and SQL formats)

In [ ]:
cols = [c.replace("'", "").replace(",", "").replace(" )", ")") for c in final_participants_df.columns]
cols = [c[1:-1] if c[0] == "(" and c[-1] == ")" else c for c in cols]
final_participants_df.columns = cols
final_participants_df.columns

Calculate the Hirabayashi Recovery Ratio (HRR) and whether it is a significant improvement or not (HRR >= 0.5)

In [ ]:
def hrr(mjoa_init, mjoa_1year):
    numerator = mjoa_1year - mjoa_init
    denominator = 18 - mjoa_init
    return numerator / denominator

In [ ]:
hrr_vals = hrr(final_participants_df['mJOA initial'], final_participants_df['mJOA 12 months'])
final_participants_df['HRR'] = hrr_vals
final_participants_df['Recovery Class'] = ['good' if v >= 0.5 else "fair" for v in hrr_vals]
final_participants_df.loc[pd.isna(hrr_vals), 'Recovery Class'] = np.nan
final_participants_df = final_participants_df.dropna(subset=['Recovery Class'])
final_participants_df.shape

Drop any patients which did not undergo surgical treatment

In [ ]:
final_participants_df = final_participants_df.loc[final_participants_df['Surgical'] == 1, :]
final_participants_df.shape

Save the results on their own for isolated ML model testing

In [ ]:
final_participants_df.to_csv('participants_cleaned.tsv', sep='\t')